### Scraping of historical data of https://pokemonprices.com/top_100 for 2019 and 2020


In [27]:
days = ["20190706", "20190726", "20190827",
        "20190928", "20191005", "20191018",
       "20191117", "20200301", "20200331", "20200428",
       "20200430", "20200520", "20200808"] # 13 days, each containing 100 records

In [28]:
urls = []
for i, day in enumerate(days):
    url = "https://web.archive.org/web/"+ day + "/https://pokemonprices.com/top_100"
    urls.append(url)
    

In [29]:
len(urls) # 13 urls

13

In [30]:
from bs4 import BeautifulSoup as bs
import urllib.request

In [31]:
name_price = [] # extracts both names and prices
prices = [] # get prices
names = [] # get names
dates = [] # dates of record
unique_nums = [] # contains unique rarity number of a pokemon card that needs some further cleaning and empty string items 
updated_nums = [] # only contains unique rarity number of a pokemon card that needs some further cleaning
nums = [] # cleaned unique rarity number of a pokemon card

for i, url in enumerate(urls):
    
    html = urllib.request.urlopen(url).read().decode('utf-8')
    soup = bs(html, 'html.parser')  
    relevant= soup.find_all('tbody')

    for price in soup.find_all('b'):
        name_price.append(price)

    for unique_num in soup.find_all('p'):
        unique_num = str(unique_num)[50:200].strip()
        unique_nums.append(unique_num)
    unique_nums = unique_nums[:-2]
        
    for date in soup.find_all('small'): # create a list of lists
        a = [date.text.strip()[15:-11].strip()] * 100 
        dates.append(a)

In [32]:
dates = [item for sublist in dates for item in sublist] # flatten the dates list

In [33]:
len(dates) # obtain a total of 1300 records

1300

In [34]:
for i, item in enumerate(name_price):
    if i % 2 == 0:
        names.append(item.text)
    else:
        prices.append(item.text[1:]) # get rid of dollar sign

In [35]:
updated_names = [] # remove any special character
for name in names:
    new_name = [character for character in name if character.isalnum()]
    new_name = "".join(new_name)
    updated_names.append(new_name)

In [37]:
updated_names[:2]

['Umbreon', 'Gastly']

In [38]:
for i,item in enumerate(unique_nums):
    if i % 2 == 0:
        updated_nums.append(item)   

         # unique card identified
for i, value in enumerate(updated_nums):
    updated = value.replace(" ", "")
    pattern = '<br/>'
    index1 = updated.find(pattern)
    updated = updated[index1+len(pattern):]
    index2 = updated.find(pattern)
    updated = updated[:index2].strip()
    nums.append(updated)

In [39]:
len(names) # double check that there are 1300 entries

1300

In [40]:
final_list = []
for i in range(len(names)):
    row=(dates[i],updated_names[i],prices[i], nums[i])
    final_list.append(row)
    

In [41]:
colnames = ['date','card_name','price', 'unique_num']

In [43]:
import pandas as pd
df = pd.DataFrame(final_list, columns=colnames)

In [44]:
df

,date,card_name,price,unique_num
0,Jul 6 2019,Umbreon,1249.95,17/17
1,Jul 6 2019,Gastly,1014.98,019/093
2,Jul 6 2019,Espeon,999.99,16/17
3,Jul 6 2019,Chansey,814.98,048/093
4,Jul 6 2019,Machop,814.98,021/093
...,...,...,...,...
1295,Aug 7 2020,Palkia,60.07,SL8
1296,Aug 7 2020,DarkHoundoom,60.00,5/109
1297,Aug 7 2020,LugiaEX,60.00,134/135
1298,Aug 7 2020,HoOh,60.00,7/64


In [45]:
df['date'] = pd.to_datetime(df['date'], format='%b %d %Y') # convert string to date

In [46]:
df

,date,card_name,price,unique_num
0,2019-07-06,Umbreon,1249.95,17/17
1,2019-07-06,Gastly,1014.98,019/093
2,2019-07-06,Espeon,999.99,16/17
3,2019-07-06,Chansey,814.98,048/093
4,2019-07-06,Machop,814.98,021/093
...,...,...,...,...
1295,2020-08-07,Palkia,60.07,SL8
1296,2020-08-07,DarkHoundoom,60.00,5/109
1297,2020-08-07,LugiaEX,60.00,134/135
1298,2020-08-07,HoOh,60.00,7/64


In [47]:
df.to_csv('rough.csv', index=False)